In this notebook we use Python to draw maps of every single country in the world, as well as a map of the EU

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#Standard python library. Manipulates Excel-like tables (called dataframes) with named rows and columns
import pandas as pd

# preparation

## country dictionary

In [3]:
iso_to_wb = pd.read_csv("https://raw.githubusercontent.com/adrivsh/country_names/master/iso3_to_wb_name.csv", index_col="iso3")
iso_to_wb.head()

,country
iso3,
ABW,Aruba
AFG,Afghanistan
AGO,Angola
AIA,Anguilla
ALB,Albania


In [4]:
from replace_with_warning import replace_with_warning

## Mapping libraries

The following library, coded for this project, allows to plot maps. It produces [SVG](https://en.wikipedia.org/wiki/Scalable_Vector_Graphics) maps, that can be visualized in a browser, out of the box. 

It can also automatically transform those SVG maps into [png](https://en.wikipedia.org/wiki/Portable_Network_Graphics) maps, provided the free softaware [Inkspace](https://inkscape.org/en/download/) is installed on your computer. 
Finally, to automatically merge the map and the legend in a single file, it requires [ImageMagick](http://www.imagemagick.org/script/index.php).




In [5]:
from maps_lib import * 

In [6]:
#ploting capacities
import matplotlib.pyplot as plt 
#make plots appear in this notebook
%matplotlib inline  

In [29]:
# svg_to_use = "FRA_guyana_ok"
svg_to_use = "Boundaries_in_asia_and_more_detail"

## Base world map

In [30]:
from tempfile import mkstemp
from shutil import move
from os import remove, close

def replace(file_path, new_path, pattern, subst):
    #Create temp file
    fh, abs_path = mkstemp()
    
    #actually replaces text line by line
    with open(abs_path,'w') as new_file:
        with open(file_path) as old_file:
            for line in old_file:
                new_file.write(line.replace(pattern, subst))
    
    #closes the temporary file
    close(fh)
    #Move temp to new file
    move(abs_path, new_path)

if svg_to_use == "Boundaries_in_asia_and_more_detail":
        #copies from map with french guyana to new one, with smaller (less thick) boundaries    
    replace("world-original.svg","world-original_thin.svg",".5px",".1px")
elif svg_to_use == "FRA_guyana_ok":    
    #copies from map with french guyana to new one, with smaller (less thick) boundaries    
    replace("world-fra.svg","world-original_thin.svg",".5px",".1px")
else:
    error, "use one of the above options"
    
    
#changes the colors of the boundaries and fill 
replace("world-original_thin.svg","world-original_thin.svg","stroke: #666","stroke: #6ec8bb")#ccc
replace("world-original_thin.svg","world-original_thin.svg","fill: #ccc","fill: #b7e1da")
replace("world-original_thin.svg","world-original_thin.svg","fill: #E0E0E0","fill: #b7e1da")

#folder where to store the pdf
outfolder_name = os.path.join(svg_to_use,"")
print(outfolder_name)
os.makedirs(outfolder_name, exist_ok=True)


Boundaries_in_asia_and_more_detail\


# Country per country and EU shadow maps

In [31]:
countries = pd.Series(0,index=iso_to_wb.index)
countries.head()

iso3
ABW    0
AFG    0
AGO    0
AIA    0
ALB    0
dtype: int64

In [ ]:
# for c in ["AND", "COK", "LIE", "MCO", "MMR", "NIU", "NRU", "SMR", "SOM", "SUR"]: #"countries that where missing first
for c in iso_to_wb.index: #all countries
    s_country = countries.copy() 
    s_country = (s_country +pd.Series({c:1})).fillna(0)
    
    spelled_out_name = iso_to_wb.ix[c].country
    
    if any([os.path.isfile(os.path.join(outfolder_name,spelled_out_name+".pdf")), 
            os.path.isfile(os.path.join(outfolder_name,"map_of_"+spelled_out_name+".pdf"))]):
           print("skipping {c}. Map already exists.".format(c=spelled_out_name))
    else:
        make_map_from_svg(
            series_in=s_country, #data 
            svg_file_path="world-original_thin.svg",    #path to blank map
    #         svg_file_path  = "world-color.svg",
            outname=spelled_out_name,           #base name for output  (will vreate img/map_of_asset_risk.png, img/legend_of_asset_risk.png, etc.)
            bins = [-1,0.5,2],
            bincolors = ["#b7e1da","#ffffff"],
            label=spelled_out_name,
            new_title="Map of the world highlighting "+spelled_out_name,#title for the colored SVG
            outfolder=outfolder_name,
            verbose=True,
            doPNG=False
        )
    
    

skipping Aruba. Map already exists 
skipping Afghanistan. Map already exists 
skipping Angola. Map already exists 
skipping Anguilla. Map already exists 
skipping Albania. Map already exists 
skipping Andorra. Map already exists 
skipping United Arab Emirates. Map already exists 
skipping Argentina. Map already exists 
skipping Armenia. Map already exists 
skipping American Samoa. Map already exists 
skipping Antigua and Barbuda. Map already exists 
skipping Australia. Map already exists 
skipping Austria. Map already exists 
skipping Azerbaijan. Map already exists 
skipping Burundi. Map already exists 
skipping Belgium. Map already exists 
skipping Benin. Map already exists 
skipping Burkina Faso. Map already exists 
skipping Bangladesh. Map already exists 
skipping Bulgaria. Map already exists 
skipping Bahrain. Map already exists 
skipping Bahamas, The. Map already exists 
skipping Bosnia and Herzegovina. Map already exists 
skipping Belarus. Map already exists 
skipping Belize. Map

## EU

In [37]:
list_of_EU = ["Austria" ,"Belgium" ,"Bulgaria" ,"Croatia" ,"Cyprus" ,"Czech Republic" ,"Denmark" ,"Estonia" ,"Finland" ,"France" ,"Germany" ,"Greece" ,
              "Hungary" ,"Ireland" ,"Italy" ,"Latvia" ,"Lithuania" ,"Luxembourg" ,"Malta" ,"Netherlands" ,
              "Poland" ,"Portugal" ,"Romania" ,"Slovakia" ,"Slovenia" ,"Spain" ,"Sweden" ,"United Kingdom" ]

any_name_to_wb_name = pd.read_csv("https://raw.githubusercontent.com/adrivsh/country_names/master/names_to_iso.csv",
                                  index_col = "country").iso3

eu_isos = any_name_to_wb_name[list_of_EU]
print(", ".join(eu_isos))


s_country = countries.copy() 
s_country = (s_country +pd.Series(1, index=eu_isos)).fillna(0)

make_map_from_svg(
        series_in=s_country, #data 
        svg_file_path="world-original_thin.svg",    #path to blank map
#         svg_file_path  = "world-color.svg",
        outname="european_union",           #base name for output  (will vreate img/map_of_asset_risk.png, img/legend_of_asset_risk.png, etc.)
        bins = [-1,0.5,2],
        bincolors = ["#b7e1da","#ffffff"],
        label="European Union",
        new_title="Map of the world highlighting European Union",
        outfolder=outfolder_name,
        verbose=True,
        doPNG=False
    )
    
    

AUT, BEL, BGR, HRV, CYP, CZE, DNK, EST, FIN, FRA, DEU, GRC, HUN, IRL, ITA, LVA, LTU, LUX, MLT, NLD, POL, PRT, ROU, SVK, SVN, ESP, SWE, GBR


# Post treatment

In [27]:
#removes "map_of_" from the filenames (if any)
import os
import glob
for f in glob.glob("*/map_of_*"):
    nu =f.replace("map_of_","")
    if os.path.isfile(nu):
        os.remove(nu)
    os.rename(f,nu)
